# Illustration of some tools

In [1]:
from stellacode.surface import IntegrationParams, FourierSurfaceFactory, CylindricalSurface
from stellacode.tools.vmec import VMECIO
from stellacode.surface.factory_tools import rotate_coil, Sequential
from stellacode.definitions import w7x_plasma, w7x_scaled_plasma
from stellacode.surface.imports import get_net_current
from stellacode.surface.current import AbstractCurrent, Current, CurrentZeroTorBC

import numpy as np
import plotly.graph_objects as go

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
n_harmonics = 4
factor = 4
rotate_diff_current = 1
make_joints = True
axis_angle = 0
sin_basis = True
cos_basis = False
build_coils = False

In [3]:
integration_par = IntegrationParams(
    num_points_u=n_harmonics * factor,
    num_points_v=n_harmonics * factor // rotate_diff_current,
)

In [4]:
surf_plasma = FourierSurfaceFactory.from_file(
    w7x_scaled_plasma.path_plasma, integration_par=integration_par)

In [5]:
net_currents = get_net_current(surf_plasma.file_path)

In [6]:
common_current_on_each_rot = False

if common_current_on_each_rot:
    current: AbstractCurrent = Current(
        num_pol=n_harmonics,
        num_tor=n_harmonics,
        sin_basis=sin_basis,
        cos_basis=cos_basis,
        net_currents=net_currents,
    )
    center_vgrid = False
else:
    current = CurrentZeroTorBC(
        num_pol=n_harmonics,
        num_tor=n_harmonics // rotate_diff_current,
        sin_basis=sin_basis,
        cos_basis=cos_basis,
        net_currents=net_currents / rotate_diff_current,
    )
    center_vgrid = True

In [7]:
integration_par = IntegrationParams(
    num_points_u=n_harmonics * factor,
    num_points_v=n_harmonics * factor // rotate_diff_current,
    center_vgrid=center_vgrid
)

In [8]:
surf_coil_0 = CylindricalSurface(
    integration_par=integration_par,
    make_joints=make_joints,
    axis_angle=axis_angle,
    nfp=surf_plasma.nfp * rotate_diff_current,
)

In [9]:
surf_coil = Sequential(
    surface_factories=[
        surf_coil_0,
        rotate_coil(
            current=current,
            nfp=surf_plasma.nfp,
            num_surf_per_period=rotate_diff_current,
            continuous_current_in_period=common_current_on_each_rot,
            build_coils=build_coils,
        ),
    ]
)

In [22]:
distance = 0.05
precision = 1e-2

In [23]:
# Get the major and minor radii of the surface
major_radius = surf_plasma().get_major_radius()
minor_radius = surf_plasma().get_minor_radius(vmec=False)

# Create a copy of the fitted surface
new_surf = surf_coil.model_copy()
radius = minor_radius + distance

# Update the parameters of the fitted surface
# The radius is set to twice the minor radius
# The distance to the origin is set to the major radius
new_surf.surface_factories[0].update_params(
    radius=radius,
    distance=major_radius,
)

# Find the minimum distance between the fitted surface and the surface
min_dist = new_surf().get_min_distance(surf_plasma().xyz)

while min_dist < distance + minor_radius*precision:
    radius = radius + min_dist
    # Update the parameters of the fitted surface
    # The radius is set to twice the minor radius
    # The distance to the origin is set to the major radius
    new_surf.surface_factories[0].update_params(
        radius=radius,
        distance=major_radius,
    )

    min_dist = new_surf().get_min_distance(surf_plasma().xyz)

In [24]:
# fig = surf_plasma().plotly_plot(reduce_res=1, colormap="Blues", nfp=5)
fig = go.Figure()
# fig2 = new_surf().plotly_plot(reduce_res=1, colormap="Reds", nfp=5)
# fig.add_trace(fig2.data[0])

trace = go.Surface(
    x=surf_plasma().xyz[..., 0],
    y=surf_plasma().xyz[..., 1],
    z=surf_plasma().xyz[..., 2],
    colorscale="Reds",
    showscale=False
)
fig.add_trace(trace)

trace = go.Surface(
    x=new_surf().xyz[..., 0],
    y=new_surf().xyz[..., 1],
    z=new_surf().xyz[..., 2],
    colorscale="Blues",
    opacity=0.5,
    showscale=False
)

fig.add_trace(trace)

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data',
    ),

    margin=dict(l=0, r=0, b=0, t=30)
)

fig.show()